# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [1]:
import cv2, imutils, re, xlsxwriter, json
import matplotlib.pyplot as plt
import numpy as np
import pytesseract
from pathlib import Path
from matplotlib import rcParams
from pytesseract import Output

In [2]:
# Directory of images to run the code on
img_dir = 'AllData/processed_images/Part2'
data_dir = 'data'

# Directory to save the output images
save_dir = 'out'

### Using json to parse the captions and dois for each image

In [3]:
with open('urldata.json') as handler:
    urldata = json.load(handler)

In [4]:
with open('doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [5]:
with open('ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

### Function to detect x-axis and y-axis for a plot

In [6]:
def findMaxConsecutiveOnes(nums) -> int:
    count = maxCount = 0
    
    for i in range(len(nums)):
        if nums[i] == 1:
            count += 1
        else:
            maxCount = max(count, maxCount)
            count = 0
                
    return max(count, maxCount)

In [7]:
def detectAxes(filepath, threshold=None, debug=False):
    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
    
    image = cv2.imread(filepath)
    height, width, channels = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[:, idx] < 200) for idx in range(width)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < width:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            break
            
        start_idx += 1
           
    yaxis = (maxindex, 0, maxindex, height)
    
    if debug:
        fig, ax = plt.subplots(1, 2)

        ax[0].imshow(image)

        ax[1].plot(maxConsecutiveOnes, color = 'k')
        ax[1].axhline(y = max(maxConsecutiveOnes) - 10, color = 'r', linestyle = 'dashed')
        ax[1].axhline(y = max(maxConsecutiveOnes) + 10, color = 'r', linestyle = 'dashed')
        ax[1].vlines(x = maxindex, ymin = 0.0, ymax = maxConsecutiveOnes[maxindex], color = 'b', linewidth = 4)

        plt.show()

    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[idx, :] < 200) for idx in range(height)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < height:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            
        start_idx += 1
            
    cv2.line(image, (0, maxindex), (width, maxindex),  (255, 0, 0), 2)
    xaxis = (0, maxindex, width, maxindex)
    
    if debug:
        rcParams['figure.figsize'] = 15, 8

        fig, ax = plt.subplots(1, 1)
        ax.imshow(image, aspect = 'auto')
        
    return xaxis, yaxis

### Here we try to get text from the image using pytesseract

In [8]:
def getTextFromImageArray(image, mode):
    image_text = []
    
    if mode == 'y-text':
        image = cv2.transpose(image)
        image = cv2.flip(image, flipCode = 1)
        config = "-l eng --oem 1 --psm 11"
    elif mode == 'y-labels':
        config = "-l eng --oem 1 --psm 6 -c tessedit_char_whitelist=.0123456789"
    
    d = pytesseract.image_to_data(image, config = config, output_type = Output.DICT)
    
    n_boxes = len(d['text'])

    # Pick only the positive confidence boxes
    for i in range(n_boxes):
            
        if int(d['conf'][i]) >= 0:
                
            text = d['text'][i].strip()
            
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            image_text.append((d['text'][i], (x, y, w, h)))
            
    # Remove all the duplicates in (text, box) pairs
    return list(set(image_text))

In [9]:
def getTextFromImage(filepath, bw=False, debug=False):
    image_text = []
    
    image = cv2.imread(filepath)
    height, width, _ = image.shape
        
    if bw:
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # define range of black color in HSV
        lower_val = np.array([0, 0, 0])
        upper_val = np.array([179, 255, 179])

        # Threshold the HSV image to get only black colors
        mask = cv2.inRange(hsv, lower_val, upper_val)

        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(image, image, mask = mask)

        # invert the mask to get black letters on white background
        image = cv2.bitwise_not(mask)
            
    d = pytesseract.image_to_data(image, config = "-l eng --oem 1 --psm 11", output_type = Output.DICT)
    n_boxes = len(d['text'])

    # Pick only the positive confidence boxes
    for i in range(n_boxes):
            
        if int(d['conf'][i]) >= 0:
                
            text = d['text'][i].strip()
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            image_text.append((d['text'][i], (x, y, w, h)))
     
    if bw:  
        image = cv2.imread(filepath)
        image_text = list(set(image_text))
        white_bg = 255 * np.ones_like(image)
        
        for text, (textx, texty, w, h) in image_text:
            roi = image[texty:texty + h, textx:textx + w]
            white_bg[texty:texty + h, textx:textx + w] = roi
            
        image_text = []
        d = pytesseract.image_to_data(white_bg, config = "-l eng --oem 1 --psm 11", output_type = Output.DICT)
        n_boxes = len(d['text'])

        # Pick only the positive confidence boxes
        for i in range(n_boxes):

            if int(d['conf'][i]) >= 0:

                text = d['text'][i].strip()
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                image_text.append((d['text'][i], (x, y, w, h)))
        
    # Remove all the duplicates in (text, box) pairs
    return list(set(image_text))

In [10]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            if bool(re.findall(r'\b[\d\.\d]+\b', text)):
                y_labels.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            if not bool(re.findall(r'\b[\d\.\d]+\b', text)):
                legends.append((text, (textx, texty, w, h)))
            
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        #cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 0), 2)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Sort bounding rects by x coordinate
    def getYFromRect(item):
        return item[1]

    maxList.sort(key = getYFromRect)
    y_labels_list = maxList.copy()
    
    x_labels = []
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
        #cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 0), 2)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    maxIntersection = 0
    maxList = []
    for i in range(y1):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    for i in range(x11, width):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    legends = []
    #legendBoxes = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
    #    legendBoxes.append((textx, texty, w, h))
        #cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)
    
    #legendBoxes = mergeRects(legendBoxes)
    
    #for (textx, texty, w, h) in legendBoxes:
    #    cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)
    
    return image, x_labels, y_labels, y_labels_list, legends

In [11]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [12]:
def maskImageForwardPass(filepath, start_idx):
    image = cv2.imread(filepath)
    height, width, channels = image.shape
        
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    start_idx = 1
    try:
        while start_idx <= width:
            if sum(gray[:, start_idx] < 200) != 0:
                break
            else:
                start_idx += 1

        end_idx = start_idx
        while end_idx <= width:
            if sum(gray[:, end_idx] < 200) == 0:
                break
            else:
                end_idx += 1
    except IndexError:
        pass

    gray[:, 1:start_idx] = 255
    gray[:, end_idx:width] = 255
        
    return gray, start_idx, end_idx

In [13]:
def maskImageBackwardPass(filepath, end_idx):
    image = cv2.imread(filepath)
    height, width, channels = image.shape

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    while end_idx > 0:
        if sum(gray[:, end_idx] < 200) == 0:
            break
        else:
            end_idx -= 1
        
    gray[:, end_idx:width] = 255
        
    return gray

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* The final ratio is: <br> ``normalize_ratio := ticks_diff.mean() / y-ticks.mean()``.

In [14]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            if bool(re.findall(r'\b[\d\.\d]+\b', text)):
                list_text.append((text, (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [15]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [16]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook

In [17]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 

Y-val data:
* The height of each bounding box is calculated by cluster count estimation method. 
* This ratio previously calculated is then used to calculate the <br> ``y_values :=  v_value = normalize_ratio x height of bounding box``.

In [18]:
def mergeRects(contours):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [19]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [20]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    #print(deltaX)
    #print(deltaY)
    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    angle = angle_between((rectAx + rectAw / 2, rectAy + rectAh / 2), (rectBx + rectBw / 2, rectBy + rectBh / 2))
    
    if (angle >= -5 and angle <= 5) or (angle <= -175 and angle >= -185) or (angle >= 175 and angle <= 185):
        if (rectBx > rectAx + rectAw):
            #print(rectA)
            #print(rectB)
            #print(angle)
            return euclidean([rectAx + rectAw / 2, rectAy + rectAh / 2], [rectBx + rectBw / 2, rectBy + rectBh / 2])
        else:
            return sys.maxsize
    else:
        return sys.maxsize

In [21]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

## Saving y-values in our data excel sheet 

In [22]:
import ast, cv2, re, sys
import math
import json


images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            xaxis, yaxis = detectAxes(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            #image_text = getTextFromImage(filepath, bw=True)
            image_text = images_text[path.name]
            img, x_labels, _, _, legends = getProbableLabels(img, image_text, xaxis, yaxis)
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')

                #rcParams['figure.figsize'] = 15, 8
                #plt.imshow(img)

                d = pytesseract.image_to_data(img, config = "-l eng --oem 1 --psm 11", output_type = Output.DICT)
                n_boxes = len(d['text'])

                # Color detection
                h, w, _ = np.shape(img)

                image = img.reshape((h * w, 3))

                # Remove white and near-by pixels
                threshold = 5
                for r in range(255 - threshold, 256):
                    for g in range(255 - threshold, 256):
                        for b in range(255 - threshold, 256):
                            image = image[np.where((image != [r, g, b]).any(axis = 1))]

                values, counts = np.unique(image, axis = 0, return_counts = True)

                greater_counts = np.where(counts >= 100)
                values, counts = values[greater_counts], counts[greater_counts]

                sort_indices = np.argsort(-counts)
                values, counts = values[sort_indices], counts[sort_indices]

                groups = []
                groupcounts = []

                for idx, value in enumerate(values):
                    grouped = False

                    for groupid, group in enumerate(groups):
                        for member in group:
                            r, g, b = member
                            vr, vg, vb = value

                            if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                                abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                                abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                                group.append(value)
                                groupcounts[groupid] += counts[idx]
                                grouped = True
                                break

                        if grouped:
                            break

                    if not grouped:
                        groups.append([value])
                        groupcounts.append(counts[idx])

                groupcounts = np.array(groupcounts)
                sort_indices = np.argsort(-groupcounts)
                new_groups = [groups[i] for i in sort_indices]
                groups = new_groups
                
                for i in range(len(groups[:4])):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    legends = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        legendtext = None

                        for text, textBox in image_text:    
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                legendtext = text

                        if closestBox is not None:
                            textBoxes.append(closestBox)

                        if legendtext is not None:
                            legends.append(legendtext)

                    list_len = []
                    
                    for rect in rects:
                        img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
                        list_len.append(float(rect[3]))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [round(l* normalize_ratio, 1) for l in list_len]

                    if path.name in yValueDict:
                        yValueDict[path.name].append([legends[-1]])
                        yValueDict[path.name].append(y_val)
                    else:
                        yValueDict[path.name] = [legends[-1]]
                        yValueDict[path.name].append(y_val)
                        
                    for box in textBoxes:
                        (x, y, w, h) = box
                        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    for (x1, y1, x2, y2) in [xaxis]:
                        cv2.line(img, (x1, y1), (x2, y2),  (0, 0, 255), 2)

                    for (x1, y1, x2, y2) in [yaxis]:
                        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        
            except Exception as e:
                print(e)
                continue
            
    return yValueDict

In [23]:
yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [45.0, 44.0, 47.0, 44.0, 46.0, 44.0], text_diff: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
[reject_outliers] ticks_diff: [45. 44. 44. 46. 44.], text_diff: [15. 15. 15. 15. 15. 15.]
[0] path: 08813216-Figure1-1.png, ratio: 0.3363228699551569

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1] path: 08813216-Figure4-1.png, ratio: nan



C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\dsand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:61: RuntimeWar

list index out of range
[get text-to-tick ratio] ticks_diff: [34.0, 34.0, 34.0, 34.0, 35.0, 33.0, 34.0, 34.0, 34.0], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [34. 34. 34. 34. 34. 34. 34.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[2] path: 1-s2.0-S0009250913008336-main-Figure10-1.png, ratio: 0.0029411764705882353

list index out of range
[get text-to-tick ratio] ticks_diff: [33.0, 34.0, 32.0, 34.0, 32.0, 35.0, 32.0, 34.0, 32.0, 33.0], text_diff: [0.1, 0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [33. 34. 34. 34. 33.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[3] path: 1-s2.0-S0009250913008336-main-Figure11-1.png, ratio: 0.002976190476190476

[get t

could not convert string to float: '120%'
[get text-to-tick ratio] ticks_diff: [250.0, 63.0], text_diff: [5.0, 20.0]
[reject_outliers] ticks_diff: [250.  63.], text_diff: [ 5. 20.]
[35] path: 1-s2.0-S0016236117305367-main-Figure4-1.png, ratio: 0.07987220447284345

[get text-to-tick ratio] ticks_diff: [43.0, 44.0, 45.0, 42.0, 44.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [43. 44. 44.], text_diff: [10. 10. 10. 10. 10.]
[36] path: 1-s2.0-S0016236117307172-main-Figure2-1.png, ratio: 0.22900763358778628

[get text-to-tick ratio] ticks_diff: [23.0, 24.0, 24.0, 24.0, 23.0, 97.0, 23.0, 24.0, 23.0], text_diff: [10.0, 10.0, 10.0, 40.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [23. 24. 24. 24. 23. 23. 24. 23.], text_diff: [10. 10. 10. 10. 10. 10. 10. 10.]
[37] path: 1-s2.0-S0016236117307172-main-Figure3-1.png, ratio: 0.425531914893617

[get text-to-tick ratio] ticks_diff: [26.0, 24.0, 24.0, 23.0, 25.0, 24.0, 25.0, 24.0, 23.0, 26.0, 24.0, 23.0, 2

[get text-to-tick ratio] ticks_diff: [31.0, 31.0, 61.0, 31.0, 31.0, 30.0], text_diff: [10.0, 10.0, 10.0, 20.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [31. 31. 31. 31. 30.], text_diff: [10. 10. 10. 10. 10.]
[66] path: 1-s2.0-S0016236118318489-main-Figure7-1.png, ratio: 0.3246753246753247

[get text-to-tick ratio] ticks_diff: [56.0, 53.0, 55.0, 162.0], text_diff: [12.0, 4.0, 4.0, 4.0]
[reject_outliers] ticks_diff: [56. 53. 55.], text_diff: [4. 4. 4.]
[67] path: 1-s2.0-S0016236118321835-main-Figure4-1.png, ratio: 0.07317073170731708

[get text-to-tick ratio] ticks_diff: [68.0, 71.0, 67.0, 69.0], text_diff: [20.0, 20.0, 20.0, 20.0]
[reject_outliers] ticks_diff: [68. 69.], text_diff: [20. 20. 20. 20.]
[68] path: 1-s2.0-S0016236119302388-main-Figure1-1.png, ratio: 0.291970802919708

[get text-to-tick ratio] ticks_diff: [76.0, 77.0], text_diff: [20.0, 20.0]
[reject_outliers] ticks_diff: [76. 77.], text_diff: [20. 20.]
[69] path: 1-s2.0-S0016236119302388-main-Figure3-1.png, ratio: 0.2614379

...

[get text-to-tick ratio] ticks_diff: [27.0, 28.0, 28.0], text_diff: [2.0, 2.0, 2.0]
[reject_outliers] ticks_diff: [28. 28.], text_diff: [2. 2. 2.]
[1094] path: ef3007992-Figure2-1.png, ratio: 0.07142857142857142

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1095] path: ef4001242-Figure2-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1096] path: ef4001242-Figure3-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [22.0, 21.0, 22.0, 64.0, 22.0, 22.0, 21.0], text_diff: [10.0, 10.0, 10.0, 30.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [22. 21. 22. 22. 22. 21.], text_diff: [10. 10. 10. 10. 10. 10.]
[1097] path: ef4001242-Figure4-1.png, ratio: 0.4615384615384615

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1098] path: ef4001242-Figure5-1.png, ratio: nan

[get text-to-tick ratio] ticks_dif

could not convert string to float: '1.00 -'
[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1134] path: energies-12-00723-Figure5-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [50.0, 50.0, 49.0, 49.0, 50.0, 100.0, 49.0], text_diff: [2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0]
[reject_outliers] ticks_diff: [50. 50. 49. 49. 50. 49.], text_diff: [2. 2. 2. 2. 2. 2.]
[1135] path: energies-12-00729-Figure2-1.png, ratio: 0.04040404040404041

[get text-to-tick ratio] ticks_diff: [45.0, 44.0, 47.0, 45.0, 44.0, 45.0, 45.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [45. 45. 45. 45.], text_diff: [10. 10. 10. 10. 10. 10. 10.]
[1136] path: energies-12-00809-v2-Figure1-1.png, ratio: 0.2222222222222222

could not convert string to float: '10%'
could not convert string to float: '30%EG'
could not convert string to float: '10%'
could not convert string to float: '30%EG'
[get text-to-tick ratio] ticks_diff

could not convert string to float: '0,12'
[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1175] path: ie061047h-Figure3-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1176] path: ie061047h-Figure9-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [39.0, 39.0, 38.0, 38.0, 40.0, 38.0, 38.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [39. 39. 38. 38. 38. 38.], text_diff: [10. 10. 10. 10. 10. 10. 10.]
[1177] path: ie100758s-Figure2-1.png, ratio: 0.2608695652173913

[get text-to-tick ratio] ticks_diff: [56.0, 54.0, 54.0, 55.0], text_diff: [20.0, 20.0, 20.0, 20.0]
[reject_outliers] ticks_diff: [54. 54. 55.], text_diff: [20. 20. 20. 20.]
[1178] path: ie100758s-Figure8-1.png, ratio: 0.36809815950920244

[get text-to-tick ratio] ticks_diff: [53.0, 162.0, 54.0], text_diff: [20.0, 60.0, 20.0]
[reject_outliers] ticks_dif

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1210] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.9-1.png, ratio: nan

list index out of range
[get text-to-tick ratio] ticks_diff: [40.0, 201.0], text_diff: [100.0, 20.0]
[reject_outliers] ticks_diff: [ 40. 201.], text_diff: [100.  20.]
[1211] path: Pan2019_Article_DistributionAndTransformationB-Figure3-1.png, ratio: 0.4979253112033195

[get text-to-tick ratio] ticks_diff: [117.0, 58.0], text_diff: [40.0, 80.0]
[reject_outliers] ticks_diff: [117.  58.], text_diff: [40. 80.]
[1212] path: Pan2019_Article_DistributionAndTransformationB-Figure6-1.png, ratio: 0.6857142857142857

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[1213] path: Phenolic Antioxidant Production by Hydrothermal Liquefaction of Lignin-Figure5-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [65.0, 193.0], text_diff: [60.0, 20.0]
[

could not convert string to float: "280'C"
[get text-to-tick ratio] ticks_diff: [31.0, 33.0, 32.0, 32.0, 33.0, 32.0, 34.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [33. 32. 32. 33. 32.], text_diff: [5. 5. 5. 5. 5. 5. 5.]
[1245] path: Tekin2016_Article_Cellulose-derivedCarbonSpheres-Figure1-1.png, ratio: 0.154320987654321

list index out of range
could not convert string to float: '(wt 9'
[get text-to-tick ratio] ticks_diff: [74.0], text_diff: [30.0]
[reject_outliers] ticks_diff: [74.], text_diff: [30.]
[1247] path: The Effect of Wood Vinegar on Hydrothermal Liquefaction of Cotton Stalk Under CO Atmosphere-Figure3-1.png, ratio: 0.40540540540540543

[get text-to-tick ratio] ticks_diff: [75.0, 76.0, 75.0, 76.0], text_diff: [10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [75. 76. 75. 76.], text_diff: [10. 10. 10. 10.]
[1248] path: Tobacco biomass as a source of advanced biofuels-Figure1-1.png, ratio: 0.13245033112582782

could not convert string 

In [24]:
workbook = xlsxwriter.Workbook('FigureData1.xlsx')

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = detectAxes(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            #rcParams['figure.figsize'] = 15, 4
            #fig, ax = plt.subplots(1, 3)

            image_text = images_text[path.name]
            image, x_labels, y_labels, y_labels_list, legends = getProbableLabels(image, image_text, xaxis, yaxis)
            
            # Make a backward pass to skip the ticks
            gray = maskImageBackwardPass(filepath, yaxis[0])

            # Processing the image so that the contours are drawn to labels and text
            ret, thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 15))
            thresh = cv2.dilate(thresh, rect_kernel, iterations = 1)
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 1))
            thresh = cv2.dilate(thresh, rect_kernel, iterations = 1)

            contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = contours[0] if len(contours) == 2 else contours[1]
            rects = [cv2.boundingRect(contour) for contour in contours]
            print("number of contours: ", len(contours))

            #image_text = getTextFromImage(filepath, bw=True)
            image_text = images_text[path.name]
            #white_bg = 255 * np.ones_like(gray.copy())

            #for text, (textx, texty, w, h) in y_labels_list:
            #    roi = gray[texty:texty + h, textx:textx + w]
            #    white_bg[texty:texty + h, textx:textx + w] = roi

            # Get y-labels
            #y_labels_list = getTextFromImageArray(white_bg, 'y-labels')
            #ax[0].imshow(white_bg, aspect = 'auto')

            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            # Whiten y-labels and get y-text
            y_labels_list.sort(key = getYFromRect)
            #y_labels = []
            for text, (textx, texty, w, h) in y_labels_list:
                roi = 255 * np.ones_like(gray[texty:texty + h, textx:textx + w])
                gray[texty:texty + h, textx:textx + w] = roi
                #y_labels.append(text)

            y_text_list = getTextFromImageArray(gray, 'y-text')

            # Sort bounding rects by x coordinate
            def getXFromRect(item):
                return item[1][0]

            y_text_list.sort(key = getXFromRect)        
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Get the remaining: x-labels and legends
            #image_text = getTextFromImage(filepath, bw=True)
            
            # Append doi values for each image.
            if path.name[:-16]+'.pdf' in doidata['Dois']:
                doi = doidata['Dois'][path.name[:-16]+'.pdf']
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            #addToExcel("caption", [urldata[path.name]], 0)
            addToExcel("doi",doi, 0)
            addToExcel("file name", [path.name[::-4] + ".pdf"], 1)
            addToExcel("x-labels", x_labels, 2)
            addToExcel("y-text", y_text, 3)
            addToExcel("y-labels", y_labels, 4)
            addToExcel("legends", legends, 5)
            
            for col_num, data in enumerate(yValueDict[path.name]):
                print(col_num, data)
                if type(data) != list:
                    worksheet.write(9 + col_num, 0, "Y-values:" + "\n\n" + str(data))
                else:
                    for pos,d in enumerate(data):
                        worksheet.write(9 + col_num, pos, str(d))

            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("Y- Values    :  ", ' '.join([str(elem) for elem in yValueDict[path.name]]), end = "\n\n")

            for (x1, y1, x2, y2) in [xaxis]:
                cv2.line(image, (x1, y1), (x2, y2),  (0, 0, 255), 2)

            for (x1, y1, x2, y2) in [yaxis]:
                cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            for text, (textx, texty, w, h) in y_labels_list:
                cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)

            #ax[1].imshow(gray, aspect = 'auto')
            #ax[2].imshow(image, aspect = 'auto')

            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: 08813216-Figure1-1.png
number of contours:  7
0 100.2
1 [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7]
2 ['0.1']
3 [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2]
4 ['Yield']
5 [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5]
6 ['A']
7 [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]
file name    :   08813216-Figure1-1.png
doi          :   
x-labels     :   ['270 0', "290'C", '310 C', "330'C", '350']
y-text       :   ['7', 'Percentage', 'o', '*', 'of', 'prod', 'o', 'lucts', '(%)', '©']
y-labels     :   ['105', '90', '75', '60', '45', '30', '15']
legends      :   ['Aqueous', 'Phase', 'Gaseous', 'Phase', 'Solid', 'Residue']
Y- Values    :   100.2 [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7] ['0.1'] [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2] ['Yield'] [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5] ['A'] [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]

[3] path: 1-s2.0-S0009250913008336-main-Figure11-1.png
number of contours:  12
0 MSP
1 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

number of contours:  9
0 1.5
1 [0.0, 0.1, 0.1, 0.1, 0.2, 0.1, 15.1, 0.1, 4.0, 0.1, 0.4, 1.3, 0.5, 0.1, 0.2, 0.1, 0.2, 0.1, 0.4, 0.1, 0.1, 0.1, 0.1, 0.4, 0.4, 4.4, 0.1, 0.5, 1.5, 0.5, 0.1, 0.2, 0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 4.1, 0.1, 1.6, 0.1, 0.1, 0.1, 3.9, 0.1, 0.1, 1.7, 0.1]
2 ['1.5']
3 [0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.6, 0.1, 0.1, 0.1, 0.2, 0.3, 0.2, 0.5, 0.5, 0.4, 0.5, 0.2, 0.5, 0.5, 0.1, 0.5, 0.0, 0.2, 0.2, 0.1, 0.2, 0.4, 0.3, 0.2, 0.4, 0.2, 0.8]
file name    :   1-s2.0-S0016236114000477-main-Figure8-1.png
doi          :   
x-labels     :   ['K2CO3', 'K2CO3', 'K2C03', 'K2C03']
y-text       :   ['e', 'nv', 'Fatty', '*', 'acid', 'content', 'oo', 'Ss', '[wt]', '5', 'zg']
y-labels     :   ['14', '12', '10', '8', '4', '2']
legends      :   []
Y- Values    :   1.5 [0.0, 0.1, 0.1, 0.1, 0.2, 0.1, 15.1, 0.1, 4.0, 0.1, 0.4, 1.3, 0.5, 0.1, 0.2, 0.1, 0.2, 0.1, 0.4, 0.1, 0.1, 0.1, 0.1, 0.4, 0.4, 4.4, 0.1, 0.5, 1.5, 0.5, 0.1, 0.2, 0.1,

number of contours:  1
0 Protein
1 [1.2, 0.2, 0.2, 0.2, 1.1, 0.4, 0.4, 0.7, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.9, 0.4, 0.2, 2.3, 0.2, 0.2, 0.2, 0.2, 2.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2, 1.2, 0.2, 0.2, 0.2, 0.2, 0.9, 0.2, 0.2, 0.2, 2.0, 0.2, 0.9, 0.2, 0.2, 0.9, 0.2, 0.2, 0.7, 0.2, 0.2, 0.2]
2 ['Protein']
3 [0.9, 0.9, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 1.2, 0.2, 0.2, 0.2, 0.7, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.7, 0.2, 0.4, 0.7, 0.9, 0.2, 0.9, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.7, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 1.1, 1.1, 0.7, 0.2, 0.2, 0.2, 0.2, 1.1, 0.2, 0.2, 0.9, 0.2, 0.2]
file name    :   1-s2.0-S0016236117301709-main-Figure1-1.png
doi          :   
x-labels     :   ['loss', 'Ash', 'Lipid', 'Protein']
y-text       :   ['Components', '(wt.%)']
y-labels     :   ['5', '50', '45', '40', '35', '30', '25', '20', '15', '10', '5', '0']
legends      :   []
Y- Values  

number of contours:  2
0 Pine:Algae
1 [0.1, 3.7, 5.7, 6.4, 4.6, 2.5, 3.6, 3.3, 2.3, 0.3, 1.3]
2 ['Pine:Algae']
3 [0.4, 0.1, 6.6, 7.4, 5.7, 3.6, 2.0, 1.0, 1.5, 0.3, 1.0, 0.1, 0.4, 0.4]
4 ['100%']
5 [0.4, 0.1, 9.6, 4.9, 2.3, 0.7, 0.1, 1.4, 4.4, 5.3, 4.1, 0.3]
file name    :   1-s2.0-S0016236117315831-main-Figure1-1.png
doi          :   
x-labels     :   ['3', 'N3O2', 'N3O4', 'N3O5', 'N4O1', 'N4O2', 'N4O3', 'NO4', 'N5', 'N5O3']
y-text       :   ['‘%', 'Relative', 'Abundance']
y-labels     :   ['14', '12', '10', '8']
legends      :   ['Pine', 'Pine:Algae', 'Pine:Algae', 'Algae']
Y- Values    :   Pine:Algae [0.1, 3.7, 5.7, 6.4, 4.6, 2.5, 3.6, 3.3, 2.3, 0.3, 1.3] ['Pine:Algae'] [0.4, 0.1, 6.6, 7.4, 5.7, 3.6, 2.0, 1.0, 1.5, 0.3, 1.0, 0.1, 0.4, 0.4] ['100%'] [0.4, 0.1, 9.6, 4.9, 2.3, 0.7, 0.1, 1.4, 4.4, 5.3, 4.1, 0.3]

[46] path: 1-s2.0-S0016236118303363-main-Figure3-1.png
number of contours:  8
0 Gas,
1 [11.7, 11.8, 3.3, 8.9]
2 ['Bio-residue']
3 [0.2, 0.2, 0.2, 0.4, 0.2, 1.2, 0.2, 0.2, 0.2, 1

number of contours:  9
0 -O
1 [nan, nan, nan]
2 ['IA-6-J,D']
3 [nan, nan, nan]
file name    :   1-s2.0-S0016236118318398-main-Figure12-1.png
doi          :   
x-labels     :   ['7.4-9.0', '7.4-6.2', '5.1-6.2', '4.8-5.1', '4.3-4.8', '2.4-4.3', '2.0-2.4', '1.09-2.0']
y-text       :   ['ot', 'Proton', '888', 'Content,', '%', '88', 'mole', 'H', 'x', '3']
y-labels     :   []
legends      :   ['---', '--U', '-O']
Y- Values    :   -O [nan, nan, nan] ['IA-6-J,D'] [nan, nan, nan]

[59] path: 1-s2.0-S0016236118318398-main-Figure4-1.png
number of contours:  13
0 Aliphatic
1 [83.9, 4.0, 70.7, 13.8, 71.6, 62.1, 2.8, 64.3, 68.3, 23.6, 45.3, 42.6, 70.7, 72.9, 71.6, 48.7, 38.3]
2 ['DRON']
3 [83.6, 4.0, 70.7, 14.1, 45.0, 42.6, 70.7, 16.5, 73.2, 71.3, 20.2]
4 ['DRON']
5 [4.6, 4.6, 4.6, 4.9, 4.9, 4.6, 4.6, 4.9, 4.6, 4.6, 1.5, 4.3]
file name    :   1-s2.0-S0016236118318398-main-Figure4-1.png
doi          :   
x-labels     :   ['FP-9-J', 'FP-11-G', 'FP-12-J', 'FP-13-G', 'FP-14-J.D']
y-text       :   ['os',

...

number of contours:  7
0 95
1 [72.4, 65.5, 49.3, 48.3, 5.5, 3.4, 51.4]
2 ['70']
3 [13.8, 5.5, 23.1, 44.5, 46.2, 41.7]
4 ['rate']
5 [12.4, 10.0, 5.5, 5.2, 4.5, 5.2]
file name    :   er.4843-Figure3-1.png
doi          :   
x-labels     :   ['25', '50', '75', '100']
y-text       :   ['8', 'w', '&', 'Yield', '(%)', '3', 'g', 'e']
y-labels     :   ['100', '80', '60', '40', '20']
legends      :   ['MPa', 'MPa', 'MPa', 'MPa', 'MPa']
Y- Values    :   95 [72.4, 65.5, 49.3, 48.3, 5.5, 3.4, 51.4] ['70'] [13.8, 5.5, 23.1, 44.5, 46.2, 41.7] ['rate'] [12.4, 10.0, 5.5, 5.2, 4.5, 5.2]

[1157] path: er.4843-Figure4-1.png
number of contours:  7
0 90
1 [0.7, 62.0, 49.5, 55.9, 5.4, 57.3]
2 ['70']
3 [24.1, 5.4, 29.8, 44.7, 37.3, 35.6]
4 ['08']
5 [23.1, 7.1, 5.4, 5.1, 6.1, 6.4]
6 ['MPa']
7 [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 1.4, 1.7, 100.7, 1.0, 2.4, 100.7, 1.7, 1.0, 1.7, 1.7, 3.1, 3.1, 1.7, 1.7, 1.7, 2.0, 1.7, 0.7, 0.7, 1.4, 1.0, 3.4, 0.7, 1.4, 2.7, 0.7, 2.4, 1.7, 0.7, 0.7, 0.7, 0.7]
file name    :   er.4

number of contours:  8
0 Water
1 [77.3, 18.4, 18.7, 2.8, 96.4, 0.7, 0.3, 0.7, 0.3, 0.3, 0.7, 88.1]
2 ['Sorbitol']
3 [0.3, 0.3, 0.3, 3.5, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1.0, 0.3, 0.3, 0.3, 0.3, 0.3, 1.0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1.7, 94.7, 0.3, 0.3, 1.7, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 2.1, 102.0, 0.3, 0.3, 0.3, 0.3, 0.3, 2.4, 0.3]
file name    :   jctb.3933-Figure5-1.png
doi          :   
x-labels     :   ['Water', 'Glycerol', 'Sorbitol']
y-text       :   ['x', 'BET', 'surface', '=', 'g', 'area', '/', 'm?-g"', 'g', '3']
y-labels     :   ['120', '60']
legends      :   []
Y- Values    :   Water [77.3, 18.4, 18.7, 2.8, 96.4, 0.7, 0.3, 0.7, 0.3, 0.3, 0.7, 88.1] ['Sorbitol'] [0.3, 0.3, 0.3, 3.5, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1.0, 0.3, 0.3, 0.3, 0.3, 0.3, 1.0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1.7, 94.7, 0.3, 0.3, 1.7, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 2.1, 102.0, 0.3, 0.3, 0.3, 0.3, 0.3, 2.4, 0.3]

[1194] path: Kumar-

number of contours:  0
0 15.72
1 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
2 ['15.72']
3 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   Onoja2019_Article_OilPalmElaeisGuineensisBiomass-Figure3-1.png
doi          :   
x-labels     :   ['EFB', 'ME', 'PKS', 'POME', 'OPE', 'OPL', 'OPT']
y-text       :   []
y-labels     :   []
legends      :   []
Y- Values    :   15.72 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

number of contours:  5
0 (%)
1 [33.4, 38.5, 34.4, 34.4, 38.5, 26.3, 42.5, 51.6, 50.6, 45.1, 17.7, 24.3, 1.0]
2 ['content']
3 [20.3, 12.2, 16.2, 26.3, 16.2, 35.9, 20.3, 26.8, 22.8, 28.4, 28.4, 24.3, 1.0]
4 ['content']
5 [19.2, 24.3, 26.3, 14.2, 20.3, 22.3, 18.2, 8.1, 6.1, 8.1, 38.5, 36.5]
file name    :   s12866-017-1144-x-Figure4-1.png
doi          :   
x-labels     :   ['TGA', 'TGAR', 'TGA', 'TGAS', 'PGA', 'PGAD', 'PBGA', 'PBGA,', 'PGA', 'REFL', 'REP,']
y-text       :   []
y-labels     :   ['120', '100', '80']
legends      :   ['IHILLLII']
Y- Values    :   (%) [33.4, 38.5, 34.4, 34.4, 38.5, 26.3, 42.5, 51.6, 50.6, 45.1, 17.7, 24.3, 1.0] ['content'] [20.3, 12.2, 16.2, 26.3, 16.2, 35.9, 20.3, 26.8, 22.8, 28.4, 28.4, 24.3, 1.0] ['content'] [19.2, 24.3, 26.3, 14.2, 20.3, 22.3, 18.2, 8.1, 6.1, 8.1, 38.5, 36.5]

[1224] path: Saber2018_Article_UltrasonicPretreatmentForLow-t-Figure3-1.png
number of contours:  8
0 ST:90
1 [17.0, 23.3, 26.3]
file name    :   Saber2018_Article_UltrasonicPretreat

number of contours:  7
0 1.7
1 [8.2, 0.3, 1.9, 0.3, 0.8, 0.7, 0.1, 0.1]
2 ['1.7']
3 [6.9, 0.7, 0.3, 1.6, 0.1, 0.9, 0.1, 0.1, 0.1, 0.1]
4 ['1.7']
5 [0.1, 5.0, 2.3, 1.1, 1.1, 0.1, 0.1, 0.1, 0.4, 0.1, 0.1, 0.1, 0.4]
file name    :   Tobacco biomass as a source of advanced biofuels-Figure1-1.png
doi          :   
x-labels     :   ['China', 'Brazil', 'India', 'USA', 'Argentina', 'Indonezia', 'Turkey', 'Greece', 'Italy', 'Pakistan']
y-text       :   ['‘Tobacco', 'leaf', 'production', '(%)', '2']
y-labels     :   ['50', '40', '30', '20', '10']
legends      :   []
Y- Values    :   1.7 [8.2, 0.3, 1.9, 0.3, 0.8, 0.7, 0.1, 0.1] ['1.7'] [6.9, 0.7, 0.3, 1.6, 0.1, 0.9, 0.1, 0.1, 0.1, 0.1] ['1.7'] [0.1, 5.0, 2.3, 1.1, 1.1, 0.1, 0.1, 0.1, 0.4, 0.1, 0.1, 0.1, 0.4]

[1250] path: wene.319-Figure5-1.png
number of contours:  5
0 reactors
1 [48.1, 6.7, 50.3, 52.6]
2 ['reators']
3 [38.0, 40.8, 6.7, 42.0]
4 ['Scenario']
5 [6.7, 14.5, 14.5, 14.5]
file name    :   wene.319-Figure5-1.png
doi          :   
x-labe